In [1]:
import os
import sys
import time
import requests

from pathlib import Path
from IPython.display import clear_output

from modules import naver
from modules.utils import iotools, timestamp

In [2]:
dataDir = Path('./data')
cookieFile = dataDir / 'naver_cafe-user.json'

articlesFileName = 'articles.json'
fileList = [_ for _ in os.listdir(dataDir) if articlesFileName in _]

# fileList.sort()
len(fileList)


11

In [3]:
headers = {'User-Agent': 'Mozilla/5.0'}
cookies = iotools.load_from_json(cookieFile)
session = requests.Session()

session.headers.update(headers)
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

print(session.cookies)


<RequestsCookieJar[<Cookie NID_AUT=hwRi/27sTdI5a8+8/ZOIWSxx2vM02q6aMODhNbmeRPyizJEYaeas8HYKDnUoLQHv for />, <Cookie NID_JKL=sXfcuA2kv7rFlRmckfdGl5+sUWB5k6vv444MpScOJBA= for />, <Cookie NID_SES=AAABfFS0E51A4p+3NOdo2WKnuS4NxIAMJhGUTOPrMDIo5UW3ikSgJkWVNwq44i2krX9TdVhlBnIzmWIjjUDdSttizS7UirQMuqMcjMHCnpH4XUQI506w2UGQHrtYBOYI+R6S0uXmAnbJeYUwNV/dxxlWvTeBytEcepykI0ud4ddM1QOf/8OFWyZDK4mLNGs6uK1hhSg4yTCZugEFLpMa1WW+f7UEmrSyr89+yK8WjQ/cBI7nLxpvLP/cOjcMsbZt36A6TW0nojdeoMqth6troNypF9UNuUPqWPdv/emCCqlsdnVGjixtrBmOdraH7mtsO9uun5NI2dMIiNTsah+ll+TgkEYbK6iJzN36cQFnEvr3ZD8Z38w75WGwhrKhOqTNFZZMoCfOP2olxyEHm1aB/s8DnN4Kvqh544ph7F6vaF/tV7Tvkw7QqQcuDTggi/CKUTvPlEatXnga0Xum6FPbbFxHeZ4iAfAV1ZolaNLarRlnaSj81XZQTvgcfHnvv7+wWh0G4w== for />, <Cookie NNB=YOLYOCZ4EDFGE for />, <Cookie nid_inf=1836321479 for />, <Cookie nid_slevel=-1 for />]>


In [4]:
for articleIDFile in fileList:
    articleIDList = iotools.load_from_json(dataDir / articleIDFile)
    cafeID, menuID = articleIDFile.split('-')[0].split('_')[2:]

    articleFile = dataDir / f'{cafeID}_{menuID}-naver_cafe-article.csv'
    commentFile = dataDir / f'{cafeID}_{menuID}-naver_cafe-comment.csv'

    total = len(articleIDList)
    tic = time.time()

    print(cafeID, menuID, total)

    for i, articleDict in enumerate(articleIDList):
        articleID = articleDict['articleId']

        url = naver.url_gen_read(cafeID, articleID)
        article = naver.CafeArticle(articleID, menuID, cafeID)

        try:
            response = session.get(url)

            if response.ok:
                data = response.json()
            result = data['result']

            article.set_from_json(result)
        except KeyboardInterrupt:
            break
        except:
            print(i)
            continue
        else:
            article.save_article(articleFile)
            article.save_comment(commentFile)
        finally:
            timestamp.progress_bar(i+1, total, time.time()-tic)


15754634 1211 20000
Percent: [------------------------------------------------->] 100%  Time: 00:5915754634 9686 1128
Percent: [------------------------------------------------->] 100%  Time: 03:1315754634 1212 20000
Percent: [------------------------------------------------->] 100%  Time: 57:1615754634 8719 20000
Percent: [------------------------------------------------->] 100%  Time: 00:3415754634 870 20000
Percent: [----------------------------------->              ] 72%  Time: 43:01

c:\Users\IPA\miniconda3\envs\ipa\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Percent: [------------------------------------------------->] 100%  Time: 59:5715754634 2642 1616
Percent: [------------------------------------------------->] 100%  Time: 04:3915754634 4830 372
Percent: [------------------------------------------------->] 100%  Time: 01:3915754634 9486 255
Percent: [------------------------------------------------->] 100%  Time: 01:01

FileNotFoundError: [Errno 2] No such file or directory: 'data\\20220711_0922_16996348_313-articles.json'